In [1]:
from train_supervision import *

In [2]:
from tools.cfg import py2cfg
config = py2cfg("config/vaihingen/dcswin.py")

../pretrain_weights/stseg_small.pth


In [34]:
import torch
import torchvision.models as models
model = Supervision_Train.load_from_checkpoint(os.path.join(config.weights_path, config.test_weights_name+'.ckpt'), config=config)


In [36]:
print(config['runtime'])

cpu


In [4]:
model.eval()


Supervision_Train(
  (net): DCSwin(
    (backbone): SwinTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
        (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (layers): ModuleList(
        (0): BasicLayer(
          (blocks): ModuleList(
            (0): SwinTransformerBlock(
              (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
              (attn): WindowAttention(
                (qkv): Linear(in_features=96, out_features=288, bias=True)
                (attn_drop): Dropout(p=0.0, inplace=False)
                (proj): Linear(in_features=96, out_features=96, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
                (softmax): Softmax(dim=-1)
              )
              (drop_path): Identity()
              (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
              (mlp): Mlp(
  

In [29]:
dummy_input = torch.randn(2, 3, 512, 512,requires_grad=True)


In [30]:
out = model(dummy_input)

In [15]:
input_names = [ "actual_input" ]
output_names = [ "output" ]


In [22]:

out.size()

torch.Size([2, 6, 512, 512])

In [43]:
torch.onnx.export(model,
                 dummy_input,
                 "dcswin.onnx",
                 input_names=input_names,
                 output_names=output_names,
                 export_params=True,
                 opset_version=14,
                 do_constant_folding=True, 
                 dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}}
                 )

In [11]:
!pip install onnxruntime


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 253.8 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 250.5 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 218.3 kB/s eta 0:00:00:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 270.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.6/532.6 kB 240.4 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [12]:
import onnxruntime as onnxrt


In [44]:
import onnx
onnx_model = onnx.load("dcswin.onnx")
# onnx_session= onnxrt.InferenceSession("dcswin.onnx")

In [45]:
onnx.checker.check_model(onnx_model)


In [46]:
import onnxruntime
ort_session = onnxruntime.InferenceSession("dcswin.onnx")


Fail: [ONNXRuntimeError] : 1 : FAIL : Load model from dcswin.onnx failed:Node (Transpose_7340) Op (Transpose) [TypeInferenceError] Invalid attribute perm {-3, -1, -2}, input shape = {0, 0, 0}

In [33]:
!pip freeze | grep onnxruntime

onnxruntime==1.13.1
